In [ ]:
from __future__ import division, print_function

import GPy
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# parameter grid
def combinations(arrays):
    """Return a single array with combinations of parameters
    
    Parameters
    ----------
    arrays - list of np.array
    
    Returns
    -------
    array - np.array
        An array that contains all combinations of the input arrays
    """
    return np.array(np.meshgrid(*arrays)).T.reshape(-1, len(arrays))

In [ ]:
def line_search_bisection(f, bound, accuracy):
    """Maximize c so that constraint fulfilled.
    
    Parameters
    ----------
    f - callable
        A function that takes a scalar value and return True if
        the constraint is fulfilled, False otherwise.
    bound - list
        Interval within which to search
    accuracy - float
        The interval up to which the algorithm shall search
        
    Returns
    -------
    c - float
        The maximum value c so that the constraint is fulfilled    
    """
    # Break if lower bound does not fulfill constraint
    if not f(bound[0]):
        return None
    
    if f(bound[1]):
        return bound[1]
    
    while bound[1] - bound[0] > accuracy:
        mean = (bound[0] + bound[1]) / 2
        
        if f(mean):
            bound[0] = mean
        else:
            bound[1] = mean
    
    return bound[0]

In [ ]:
def lqr(A, B, Q, R):
    """
    Compute the continuous time LQR-controller. 
    
    Parameters
    ----------
    A - np.array
    B - np.array
    Q - np.array
    R - np.array
     
    Returns
    -------
    K - np.array
        Controller matrix
    P - np.array
        Cost to go matrix
    """
 
    #first, try to solve the ricatti equation
    P = sp.linalg.solve_continuous_are(A, B, Q, R)
     
    #compute the LQR gain
    K = np.linalg.solve(R, B.T.dot(P))
     
    return K, P


def quadratic_lyapunov_function(x, P):
    """
    Compute V(x) for quadratic Lyapunov function
    
    V(x) = x.T P x
    
    Equivalent, but slower implementation:
    np.array([ xi.dot(p.dot(xi.T)) for xi in x])
    
    Parameters
    ----------
    x - np.array
        2d array that has a vector x on each row
    P - np.array
        2d cost matrix for lyapunov function

    Returns
    -------
    V - np.array
        1d array with V(x)
    dV - np.array
        2d array with dV(x)/dx on each row
    """
    return np.sum(x.dot(P) * x, axis=1), x.dot(P)

In [ ]:
def get_safe_set(dV, gp_mean, gp_var, threshold, S0=None, beta=2.):
    """
    Compute the safe set
    
    Parameters
    ----------
    grid - np.array
        The states at which to compute the safe set
    dV - np.array
        The derivatives of the Lyapunov function at grid points
    gp_mean - np.array
        gp mean of the dynamics (including prior dynamics as mean)
    gp_var - np.array
        gp var of the dynamics
    threshold - float
        The safety threshold, in the paper threshold = tau * L
    S0 - np.array
        The deterministic safe set
    beta - float
        The confidence interval for the GP-prediction
    """    
    # V_dot_mean = dV * mu
    # V_dot_var = sum_i(|dV_i| * var_i)
    V_dot = np.sum(dV * gp_mean, axis=1) + beta * np.sqrt(np.sum(np.abs(dV) * gp_var, axis=1)) 
    
    if S0 is None:
        return V_dot < threshold
    else:
        return np.logical_or(V_dot < threshold, S0)
        
def find_max_levelset(S, V, accuracy, interval=None):
    """
    Find maximum level set of V in S.
    
    Parameters
    ----------
    S - boolean array
        Elements are True if V_dot <= L tau
    V - np.array
        1d array with values of Lyapunov function.
    accuracy - float
        The accuracy up to which the level set is computed
    interval - list
        Interval within which the level set is search. Defaults
        to [0, max(V) + accuracy]
        
    Returns
    -------
    c - float
        The value of the maximum level set
    """
    
    def levelset_is_safe(c):
        """
        Return true if V(c) is subset of S
        
        Parameters
        ----------
        c: float
            The level set value
            
        Returns:
        safe: boolean
        """
        return np.all(S[V <= c])
    
    if interval is None:
        interval = [0, np.max(V) + accuracy]
    return line_search_bisection(levelset_is_safe,
                                 interval,
                                 accuracy)
        

In [ ]:
tau = 0.01

# x_min, x_max, n_samples
grid_param = [(-1, 1, tau),
              (-1, 1, tau)]

grid = combinations([np.arange(*x) for x in grid_param])

In [ ]:
n = 2
m = 1

A = np.array([[0, 1],
              [0, 0]])

B = np.array([[0],
              [1]])

Q = np.diag([1, 0.01])
R = np.array([[0.1]])

K, P = lqr(A, B, Q, R)

def control_law(x):
    return -x.dot(K.T)

def true_dynamics(x):
    x = np.asarray(x)
    u = control_law(x)
    np.clip(u, -0.0001, 0.0001, out=u)
    return x.dot(A.T) + u.dot(B.T)

def prior_dynamics(x):
    x = np.asarray(x)
    u = control_law(x)
    return x.dot(A.T) + u.dot(B.T)

# Initial safe set
S0 = np.linalg.norm(grid, axis=1) < 0.1

if not np.any(S0):
    print('No initial safe points!')
    
mf = GPy.core.Mapping(2, 2)
mf.f = prior_dynamics
mf.update_gradients = lambda a,b: None

In [ ]:
kernel = GPy.kern.RBF(input_dim=2, lengthscale=0.05, variance=1)
likelihood = GPy.likelihoods.Gaussian(variance=0.00001**2)
gp = GPy.core.GP(np.array([[0, 0]]), np.array([[0, 0]]), kernel, likelihood, mean_function=mf)

L = 2 * gp.kern.variance / gp.kern.lengthscale

In [ ]:
V, dV = quadratic_lyapunov_function(grid, P)
    
def update_gp():
    mu, var = gp._raw_predict(grid)
    S = get_safe_set(dV, mu, var, L*tau, S0=S0, beta=2.)
    c = find_max_levelset(S, V, 0.001)
    S[:] = V <= c
    max_id = np.argmax(var[S])
    max_state = grid[S][max_id].copy()
    gp.set_XY(np.vstack([gp.X, max_state]),
              np.vstack([gp.Y, true_dynamics(max_state)]))
    return S
    

In [ ]:
S = np.sum(dV * true_dynamics(grid), axis=1) < L*tau
c = find_max_levelset(S, V, 0.001)
S[:] = V <= c
plt.imshow(np.reshape(S, (200, 200)))

In [ ]:
for i in range(10):
    update_gp()
S = update_gp()
print(np.count_nonzero(S))

In [ ]:
plt.imshow(np.reshape(S, (200, 200)))